In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import numpy as np
import math, copy

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by the num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attention_weights = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.d_k)
        if mask is not None:
          attention_weights = attention_weights.masked_fill(mask == 0, -1e9)
        attention_weights = F.softmax(attention_weights, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(Encoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.positional_encoding = PositionalEncoding(d_model)
        self.positional_ff = PositionWiseFeedForward(d_model, d_ff)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        encoded_data = self.positional_encoding(x)
        mha_output = self.mha(encoded_data, encoded_data, encoded_data, mask)
        x = self.norm(encoded_data + mha_output)
        ff = self.positional_ff(x)
        x = self.norm(x+ff)
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(Decoder, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model)
        self.positional_ff = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        encoded_data = self.positional_encoding(x)
        mha1_output = self.mha1(encoded_data, encoded_data, encoded_data, tgt_mask)
        x = self.norm1(encoded_data + mha1_output)
        mha2_output = self.mha2(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + mha2_output)
        ff = self.positional_ff(x)
        x = self.norm3(x + ff)
        return x


In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_len, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)
        self.encoder_layers = nn.ModuleList([Encoder(d_model, num_heads, d_ff) for i in range(num_layers)])
        self.decoder_layers = nn.ModuleList([Decoder(d_model, num_heads, d_ff) for i in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [ ]:
print(transformer.parameters)

<bound method Module.parameters of Transformer(
  (encoder_embedding): Embedding(5000, 512)
  (decoder_embedding): Embedding(5000, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x Encoder(
      (mha): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (positional_encoding): PositionalEncoding()
      (positional_ff): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x Decoder(
      (positional_encoding): PositionalEncoding(

In [ ]:
# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.68054485321045
Epoch: 2, Loss: 8.57895278930664
Epoch: 3, Loss: 8.50938892364502
Epoch: 4, Loss: 8.457099914550781
Epoch: 5, Loss: 8.410348892211914
Epoch: 6, Loss: 8.364774703979492
Epoch: 7, Loss: 8.316691398620605
Epoch: 8, Loss: 8.265087127685547
Epoch: 9, Loss: 8.211661338806152
Epoch: 10, Loss: 8.158880233764648
Epoch: 11, Loss: 8.106765747070312
Epoch: 12, Loss: 8.05601978302002
Epoch: 13, Loss: 8.006163597106934
Epoch: 14, Loss: 7.955199718475342
Epoch: 15, Loss: 7.904446125030518
Epoch: 16, Loss: 7.852644443511963
Epoch: 17, Loss: 7.801174640655518
Epoch: 18, Loss: 7.747891426086426
Epoch: 19, Loss: 7.696301460266113
Epoch: 20, Loss: 7.644042491912842
Epoch: 21, Loss: 7.5910797119140625
Epoch: 22, Loss: 7.537935733795166
Epoch: 23, Loss: 7.484330177307129
Epoch: 24, Loss: 7.430030345916748
Epoch: 25, Loss: 7.3747878074646
Epoch: 26, Loss: 7.318397521972656
Epoch: 27, Loss: 7.2605438232421875
Epoch: 28, Loss: 7.202023506164551
Epoch: 29, Loss: 7.14306402206420